In [1]:
BATCH_SIZE = 32
MULTILABEL = True
USE_TEMPLATE = True
USE_PREPROCESSING = True

In [2]:
import re
import numpy as np
import pandas as pd
from ktrain import text
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, classification_report

In [3]:
df_test = pd.read_csv('./Indonesian-Twitter-Emotion-Dataset/Twitter_Emotion_Dataset.csv')
X_test = df_test['tweet'].tolist()
y_test = df_test['label']
label = ['kemarahan', 'ketakutan', 'kebahagiaan', 'cinta', 'kesedihan']

hypotheses_template = "{}."
if USE_TEMPLATE:
    hypotheses_template = "Kalimat ini mengekspresikan {}."

if USE_PREPROCESSING:
    X_test = [re.sub(r'\[([A-Z]+)\]', r'', X) for X in X_test]
    X_test = [re.sub(r'#(\S+)', r'\g<1>', X) for X in X_test]
    X_test = [re.sub(r'(\.)\1{2,}', '', X) for X in X_test]
    X_test = [re.sub(r'(\W)\1{2,}', r'\g<1>', X) for X in X_test]
    X_test = [X.lower() for X in X_test]

In [4]:
le = LabelEncoder()
y_test = le.fit_transform(y_test)

In [5]:
zsl = text.ZeroShotClassifier('joeddav/xlm-roberta-large-xnli')

y_pred = zsl.predict(
    X_test, nli_template=hypotheses_template,
    topic_strings=label, multilabel=MULTILABEL, batch_size=BATCH_SIZE
)
y_pred = np.argmax(np.array(y_pred), axis=-1)

In [6]:
print('F1 Score:', f1_score(y_test, y_pred, average='micro'))
print(classification_report(y_test, y_pred, target_names=le.classes_))

F1 Score: 0.620540786184958
              precision    recall  f1-score   support

       anger       0.62      0.55      0.58      1101
        fear       0.51      0.81      0.63       649
       happy       0.77      0.61      0.68      1017
        love       0.62      0.81      0.70       637
     sadness       0.61      0.47      0.53       997

    accuracy                           0.62      4401
   macro avg       0.63      0.65      0.62      4401
weighted avg       0.64      0.62      0.62      4401

